## রিগ্রেশন পরিচিতি - পাঠ ১

#### বিষয়টি বুঝে নেওয়া

✅ রিগ্রেশনের অনেক ধরণ রয়েছে, এবং আপনি কোনটি বেছে নেবেন তা নির্ভর করে আপনার প্রশ্নের উত্তর কী হতে পারে তার উপর। যদি আপনি কোনো নির্দিষ্ট বয়সের ব্যক্তির সম্ভাব্য উচ্চতা পূর্বানুমান করতে চান, তাহলে আপনি `লিনিয়ার রিগ্রেশন` ব্যবহার করবেন, কারণ আপনি একটি **সংখ্যামূলক মান** খুঁজছেন। যদি আপনি জানতে চান কোনো খাবারকে ভেগান হিসেবে বিবেচনা করা উচিত কিনা, তাহলে আপনি একটি **শ্রেণী নির্ধারণ** খুঁজছেন, সেক্ষেত্রে আপনি `লজিস্টিক রিগ্রেশন` ব্যবহার করবেন। আপনি পরে লজিস্টিক রিগ্রেশন সম্পর্কে আরও শিখবেন। ডেটা থেকে কিছু প্রশ্ন করার কথা ভাবুন এবং কোন পদ্ধতিটি বেশি উপযুক্ত হবে তা নির্ধারণ করুন।

এই অংশে, আপনি [ডায়াবেটিস সম্পর্কিত একটি ছোট ডেটাসেট](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html) নিয়ে কাজ করবেন। কল্পনা করুন যে আপনি ডায়াবেটিস রোগীদের জন্য একটি চিকিৎসা পরীক্ষা করতে চান। মেশিন লার্নিং মডেলগুলো আপনাকে সাহায্য করতে পারে কোন রোগী চিকিৎসায় ভালো সাড়া দেবে তা নির্ধারণ করতে, বিভিন্ন ভেরিয়েবলের সংমিশ্রণের উপর ভিত্তি করে। এমনকি একটি খুব সাধারণ রিগ্রেশন মডেল, যখন ভিজুয়ালাইজ করা হয়, তখন এমন তথ্য দেখাতে পারে যা আপনাকে তাত্ত্বিক ক্লিনিকাল ট্রায়ালগুলো সংগঠিত করতে সাহায্য করবে।

তাহলে, চলুন এই কাজটি শুরু করা যাক!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>চিত্রকর্ম: @allison_horst</figcaption>

<!--![চিত্রকর্ম: \@allison_horst](../../../../../../translated_images/encouRage.e75d5fe0367fb9136b78104baf4e2032a7622bc42a2bc34c0ad36c294eeb83f5.bn.jpg)<br>চিত্রকর্ম: @allison_horst-->


## 1. আমাদের টুল সেট লোড করা

এই কাজের জন্য, আমাদের নিম্নলিখিত প্যাকেজগুলো প্রয়োজন হবে:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) হলো একটি [R প্যাকেজের সংগ্রহ](https://www.tidyverse.org/packages) যা ডেটা সায়েন্সকে দ্রুত, সহজ এবং আরও মজাদার করে তোলে!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) ফ্রেমওয়ার্ক হলো মডেলিং এবং মেশিন লার্নিংয়ের জন্য একটি [প্যাকেজের সংগ্রহ](https://www.tidymodels.org/packages/)।

আপনি এগুলো নিম্নলিখিতভাবে ইনস্টল করতে পারেন:

`install.packages(c("tidyverse", "tidymodels"))`

নিচের স্ক্রিপ্টটি পরীক্ষা করে দেখবে যে আপনার এই মডিউল সম্পন্ন করার জন্য প্রয়োজনীয় প্যাকেজগুলো আছে কিনা এবং যদি কিছু প্যাকেজ অনুপস্থিত থাকে তবে সেগুলো আপনার জন্য ইনস্টল করবে।


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



এখন, চলুন এই চমৎকার প্যাকেজগুলো লোড করি এবং সেগুলোকে আমাদের বর্তমান R সেশনে উপলব্ধ করি। (এটি শুধুমাত্র উদাহরণের জন্য, `pacman::p_load()` ইতিমধ্যেই আপনার জন্য এটি করেছে)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## ২. ডায়াবেটিস ডেটাসেট

এই অনুশীলনে, আমরা ডায়াবেটিস ডেটাসেটের উপর পূর্বাভাস তৈরি করে আমাদের রিগ্রেশন দক্ষতা প্রদর্শন করব। [ডায়াবেটিস ডেটাসেট](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt)-এ ডায়াবেটিস সম্পর্কিত `৪৪২টি নমুনা` রয়েছে, যেখানে ১০টি পূর্বাভাসকারী ফিচার ভেরিয়েবল রয়েছে: `বয়স`, `লিঙ্গ`, `বডি মাস ইনডেক্স`, `গড় রক্তচাপ`, এবং `ছয়টি রক্তের সিরাম পরিমাপ`। এছাড়াও একটি আউটকাম ভেরিয়েবল `y` রয়েছে: যা বেসলাইন থেকে এক বছর পর রোগের অগ্রগতির একটি পরিমাণগত পরিমাপ।

|পর্যবেক্ষণের সংখ্যা|৪৪২|
|-------------------|:---|
|পূর্বাভাসকারীর সংখ্যা|প্রথম ১০টি কলাম সংখ্যাসূচক পূর্বাভাসকারী|
|আউটকাম/টার্গেট|১১তম কলাম বেসলাইন থেকে এক বছর পর রোগের অগ্রগতির একটি পরিমাণগত পরিমাপ|
|পূর্বাভাসকারীর তথ্য|- বয়স (বছরে)
||- লিঙ্গ
||- bmi বডি মাস ইনডেক্স
||- bp গড় রক্তচাপ
||- s1 tc, মোট সিরাম কোলেস্টেরল
||- s2 ldl, লো-ডেনসিটি লিপোপ্রোটিন
||- s3 hdl, হাই-ডেনসিটি লিপোপ্রোটিন
||- s4 tch, মোট কোলেস্টেরল / HDL
||- s5 ltg, সম্ভবত সিরাম ট্রাইগ্লিসারাইডের লগ স্তর
||- s6 glu, রক্তে শর্করার স্তর|

> 🎓 মনে রাখবেন, এটি সুপারভাইজড লার্নিং, এবং আমাদের একটি নামকৃত 'y' টার্গেট প্রয়োজন।

আপনি R দিয়ে ডেটা ম্যানিপুলেট করার আগে, আপনাকে ডেটা R-এর মেমরিতে ইমপোর্ট করতে হবে, অথবা ডেটার সাথে সংযোগ তৈরি করতে হবে যাতে R দূরবর্তীভাবে ডেটা অ্যাক্সেস করতে পারে।

> [readr](https://readr.tidyverse.org/) প্যাকেজ, যা Tidyverse-এর অংশ, দ্রুত এবং সহজ উপায়ে আয়তাকার ডেটা R-এ পড়ার জন্য একটি বন্ধুত্বপূর্ণ পদ্ধতি প্রদান করে।

এখন, এই সোর্স URL থেকে ডায়াবেটিস ডেটাসেট লোড করি: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

এছাড়াও, আমরা আমাদের ডেটার উপর একটি স্যানিটি চেক করব `glimpse()` ব্যবহার করে এবং প্রথম ৫টি সারি প্রদর্শন করব `slice()` দিয়ে।

আরো এগোনোর আগে, চলুন এমন একটি বিষয় পরিচয় করিয়ে দিই যা আপনি R কোডে প্রায়ই দেখবেন 🥁🥁: পাইপ অপারেটর `%>%`

পাইপ অপারেটর (`%>%`) একটি বস্তুকে একটি ফাংশন বা কল এক্সপ্রেশনে ফরোয়ার্ড পাস করে যৌক্তিক ক্রমে অপারেশন সম্পাদন করে। আপনি পাইপ অপারেটরকে আপনার কোডে "এবং তারপর" বলার মতো ভাবতে পারেন।


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` আমাদের দেখায় যে এই ডেটাসেটে ৪৪২টি সারি এবং ১১টি কলাম রয়েছে, যেখানে সব কলামের ডেটা টাইপ `double`।

<br>

> glimpse() এবং slice() হলো [`dplyr`](https://dplyr.tidyverse.org/) প্যাকেজের ফাংশন। Dplyr, যা Tidyverse-এর অংশ, একটি ডেটা ম্যানিপুলেশনের ব্যাকরণ যা একটি সুনির্দিষ্ট সেটের ক্রিয়াপদ সরবরাহ করে, যা আপনাকে সবচেয়ে সাধারণ ডেটা ম্যানিপুলেশনের চ্যালেঞ্জগুলি সমাধান করতে সাহায্য করে।

<br>

এখন যেহেতু আমাদের কাছে ডেটা রয়েছে, আসুন এই অনুশীলনের জন্য একটি নির্দিষ্ট ফিচার (`bmi`) বেছে নিই। এর জন্য আমাদের প্রয়োজনীয় কলামগুলো নির্বাচন করতে হবে। তাহলে, আমরা কীভাবে এটি করব?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) আমাদের একটি ডেটা ফ্রেম থেকে কলাম *নির্বাচন* (এবং প্রয়োজনে পুনঃনামকরণ) করার সুযোগ দেয়।


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## ৩. প্রশিক্ষণ এবং পরীক্ষার ডেটা

সুপারভাইজড লার্নিং-এ সাধারণত ডেটাকে দুইটি উপসেটে ভাগ করা হয়; একটি (সাধারণত বড়) সেট দিয়ে মডেল প্রশিক্ষণ দেওয়া হয়, এবং একটি ছোট "হোল্ড-ব্যাক" সেট দিয়ে দেখা হয় মডেলটি কেমন কাজ করেছে।

এখন যেহেতু আমাদের ডেটা প্রস্তুত, আমরা দেখতে পারি একটি মেশিন এই ডেটাসেটে সংখ্যাগুলোর মধ্যে একটি যৌক্তিক বিভাজন নির্ধারণ করতে সাহায্য করতে পারে কিনা। আমরা [rsample](https://tidymodels.github.io/rsample/) প্যাকেজটি ব্যবহার করতে পারি, যা Tidymodels ফ্রেমওয়ার্কের অংশ, একটি অবজেক্ট তৈরি করতে যা ডেটা কীভাবে ভাগ করা হবে তার তথ্য ধারণ করে। এরপর, তৈরি করা প্রশিক্ষণ এবং পরীক্ষার সেটগুলো বের করতে আরও দুটি rsample ফাংশন ব্যবহার করা যায়:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Tidymodels ব্যবহার করে একটি লিনিয়ার রিগ্রেশন মডেল প্রশিক্ষণ দিন

এখন আমরা আমাদের মডেল প্রশিক্ষণের জন্য প্রস্তুত!

Tidymodels-এ, আপনি `parsnip()` ব্যবহার করে তিনটি ধারণা নির্ধারণ করে মডেল তৈরি করেন:

-   মডেলের **ধরন** বিভিন্ন মডেল যেমন লিনিয়ার রিগ্রেশন, লজিস্টিক রিগ্রেশন, ডিসিশন ট্রি মডেল ইত্যাদি আলাদা করে।

-   মডেলের **মোড** সাধারণ বিকল্প যেমন রিগ্রেশন এবং ক্লাসিফিকেশন অন্তর্ভুক্ত করে; কিছু মডেল ধরন উভয় মোড সমর্থন করে, আবার কিছু শুধুমাত্র একটি মোড সমর্থন করে।

-   মডেলের **ইঞ্জিন** হলো সেই গণনামূলক টুল যা মডেল ফিট করার জন্য ব্যবহৃত হবে। প্রায়ই এগুলো R প্যাকেজ হয়, যেমন **`"lm"`** বা **`"ranger"`**

এই মডেলিং তথ্য একটি মডেল স্পেসিফিকেশনে ধারণ করা হয়, তাই চলুন একটি তৈরি করি!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

একটি মডেল *নির্ধারণ* করার পর, মডেলটি [`fit()`](https://parsnip.tidymodels.org/reference/fit.html) ফাংশন ব্যবহার করে `অনুমান` বা `প্রশিক্ষণ` করা যেতে পারে, সাধারণত একটি ফর্মুলা এবং কিছু ডেটা ব্যবহার করে।

`y ~ .` এর অর্থ হলো আমরা `y`-কে পূর্বাভাসিত পরিমাণ/লক্ষ্য হিসেবে ফিট করব, যা সমস্ত পূর্বাভাসকারী/বৈশিষ্ট্য দ্বারা ব্যাখ্যাত হবে অর্থাৎ, `.` (এই ক্ষেত্রে, আমাদের কাছে শুধুমাত্র একটি পূর্বাভাসকারী আছে: `bmi`)


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

মডেলের আউটপুট থেকে আমরা প্রশিক্ষণের সময় শেখা কোয়েফিসিয়েন্টগুলো দেখতে পারি। এগুলো হলো সেরা ফিট লাইনের কোয়েফিসিয়েন্ট, যা আমাদের প্রকৃত এবং পূর্বাভাসিত ভেরিয়েবলের মধ্যে সামগ্রিক ত্রুটি সর্বনিম্ন করে।

<br>

## ৫. টেস্ট সেটে পূর্বাভাস তৈরি করুন

এখন যেহেতু আমরা একটি মডেল প্রশিক্ষণ করেছি, আমরা এটি ব্যবহার করে টেস্ট ডেটাসেটের জন্য রোগের অগ্রগতি y পূর্বাভাস করতে পারি [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html) ব্যবহার করে। এটি ডেটা গ্রুপগুলোর মধ্যে লাইন আঁকার জন্য ব্যবহৃত হবে।


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

ইয়াহু! 💃🕺 আমরা একটি মডেল প্রশিক্ষণ করেছি এবং এটি ব্যবহার করে পূর্বাভাস তৈরি করেছি!

পূর্বাভাস তৈরি করার সময়, tidymodels এর নিয়ম হলো সবসময় একটি টিবল/ডেটা ফ্রেম তৈরি করা যেখানে ফলাফলের জন্য মানসম্মত কলামের নাম থাকে। এটি মূল ডেটা এবং পূর্বাভাসগুলোকে একত্রিত করে একটি ব্যবহারযোগ্য ফরম্যাটে পরবর্তী কাজের জন্য যেমন প্লটিং-এর জন্য সহজ করে তোলে।

`dplyr::bind_cols()` দক্ষতার সাথে একাধিক ডেটা ফ্রেমের কলাম একত্রিত করে।


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## ৬. মডেলিং ফলাফল চিত্রায়িত করা

এখন, এটি ভিজ্যুয়ালি দেখার সময় 📈। আমরা টেস্ট সেটের সমস্ত `y` এবং `bmi` মানের একটি স্ক্যাটার প্লট তৈরি করব, তারপর পূর্বাভাস ব্যবহার করে মডেলের ডেটা গ্রুপিংয়ের মধ্যে সবচেয়ে উপযুক্ত স্থানে একটি রেখা আঁকব।

R-এ গ্রাফ তৈরির জন্য বেশ কয়েকটি সিস্টেম রয়েছে, তবে `ggplot2` সবচেয়ে সুন্দর এবং সবচেয়ে বহুমুখী। এটি আপনাকে **স্বতন্ত্র উপাদানগুলো একত্রিত করে** গ্রাফ তৈরি করতে দেয়।


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ এখানে কী ঘটছে তা একটু ভাবুন। একটি সরল রেখা অনেক ছোট ছোট ডেটা বিন্দুর মধ্য দিয়ে যাচ্ছে, কিন্তু এটি ঠিক কী করছে? আপনি কি দেখতে পাচ্ছেন যে এই রেখাটি ব্যবহার করে কীভাবে একটি নতুন, অদেখা ডেটা পয়েন্টের অবস্থান প্লটের y অক্ষের সাথে সম্পর্কিতভাবে পূর্বাভাস করা উচিত? এই মডেলের ব্যবহারিক দিকটি শব্দে প্রকাশ করার চেষ্টা করুন।

অভিনন্দন, আপনি আপনার প্রথম লিনিয়ার রিগ্রেশন মডেল তৈরি করেছেন, এর মাধ্যমে একটি পূর্বাভাস তৈরি করেছেন এবং এটি একটি প্লটে প্রদর্শন করেছেন!



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিক অনুবাদ প্রদানের চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা তার জন্য দায়বদ্ধ থাকব না।
